In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.0 MB/s eta 0:00:00


In [2]:
!pip install scikeras

In [3]:
 # Install the scikeras library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, accuracy_score, precision_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier # Now you can import KerasClassifier
from tensorflow import keras
from tensorflow.keras import layers
import kerastuner as kt
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

## **importing dataset**

In [4]:
data_train=pd.read_csv('8_family_training.csv')
data_test=pd.read_csv('8_family_testing.csv')
data_validation=pd.read_csv('8_family_validation.csv')

In [5]:
data_train.head()

,Malware,Executable_name,NtSetInformationProcess,NtAllocateVirtualMemory,NtProtectVirtualMemory,NtOpenFile,NtQueryInformationProcess,NtOpenKey,NtQueryValueKey,NtClose,...,1042.dll,1041.dll,1049.dll,1040.dll,1036.dll,3082.dll,1031.dll,1028.dll,2052.dll,rtinfo.dll
0,Benign,Benign_3071,0.0,13.0,11.0,6.0,13.0,3.0,9.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Benign,Benign_2017,179.0,780.0,729.0,93.0,355.0,63.0,1334.0,1755.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,qbot,backdoor_qbot_2020_124,54.0,400.0,482.0,104.0,286.0,31.0,412.0,1081.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,emotet,spyware_emotet_2019_223,12.0,146.0,197.0,71.0,116.0,223.0,467.0,77942.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Benign,Benign_2048,0.0,12.0,37.0,6.0,7.0,7.0,4.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
count=data_train['Malware'].value_counts()
print(count)

Malware
Benign    1019
qbot       509
fareit     486
shade      310
emotet     298
ursnif     254
gozi       158
banbra     150
tofsee     105
Name: count, dtype: int64


In [7]:
count_test=data_test['Malware'].value_counts()
print(count_test)

Malware
Benign    127
qbot       64
fareit     60
shade      39
emotet     37
ursnif     31
gozi       20
banbra     18
tofsee     13
Name: count, dtype: int64


In [8]:
count_validation=data_validation['Malware'].value_counts()
print(count_validation)

Malware
Benign    128
qbot       64
fareit     61
shade      39
emotet     38
ursnif     32
gozi       20
banbra     19
tofsee     13
Name: count, dtype: int64


In [9]:
X_train=data_train.drop(['Malware'],axis=1)
y_trian=data_train['Malware']

In [10]:
X_test=data_test.drop(['Malware'],axis=1)
y_test=data_test['Malware']

In [11]:
X_validation=data_validation.drop(['Malware'],axis=1)
y_validation=data_validation['Malware']

In [12]:
X_train.head()

,Executable_name,NtSetInformationProcess,NtAllocateVirtualMemory,NtProtectVirtualMemory,NtOpenFile,NtQueryInformationProcess,NtOpenKey,NtQueryValueKey,NtClose,NtQuerySystemInformation,...,1042.dll,1041.dll,1049.dll,1040.dll,1036.dll,3082.dll,1031.dll,1028.dll,2052.dll,rtinfo.dll
0,Benign_3071,0.0,13.0,11.0,6.0,13.0,3.0,9.0,32.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Benign_2017,179.0,780.0,729.0,93.0,355.0,63.0,1334.0,1755.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,backdoor_qbot_2020_124,54.0,400.0,482.0,104.0,286.0,31.0,412.0,1081.0,120.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,spyware_emotet_2019_223,12.0,146.0,197.0,71.0,116.0,223.0,467.0,77942.0,61.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Benign_2048,0.0,12.0,37.0,6.0,7.0,7.0,4.0,29.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **Preprocessing**

In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

lable = LabelEncoder()


for i in X_train.columns:
  if X_train[i].dtype == 'object':
    X_train[i] = lable.fit_transform(X_train[i])

In [14]:
X_train.head()

,Executable_name,NtSetInformationProcess,NtAllocateVirtualMemory,NtProtectVirtualMemory,NtOpenFile,NtQueryInformationProcess,NtOpenKey,NtQueryValueKey,NtClose,NtQuerySystemInformation,...,1042.dll,1041.dll,1049.dll,1040.dll,1036.dll,3082.dll,1031.dll,1028.dll,2052.dll,rtinfo.dll
0,864,0.0,13.0,11.0,6.0,13.0,3.0,9.0,32.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12,179.0,780.0,729.0,93.0,355.0,63.0,1334.0,1755.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1261,54.0,400.0,482.0,104.0,286.0,31.0,412.0,1081.0,120.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2621,12.0,146.0,197.0,71.0,116.0,223.0,467.0,77942.0,61.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,37,0.0,12.0,37.0,6.0,7.0,7.0,4.0,29.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
for j in X_test.columns:
  if X_test[j].dtype == 'object':
    X_test[j] = lable.fit_transform(X_test[j])

In [16]:
for k in X_validation.columns:
  if X_validation[k].dtype == 'object':
    X_validation[k] = lable.fit_transform(X_validation[k])

In [17]:
y_train=lable.fit_transform(y_trian)
y_test=lable.fit_transform(y_test)
y_validation=lable.fit_transform(y_validation)

In [18]:
label_mapping_train = dict(zip(range(len(lable.classes_)), lable.classes_))


# Display the mapping
print("Label Mapping:", label_mapping_train)

Label Mapping: {0: 'Benign', 1: 'banbra', 2: 'emotet', 3: 'fareit', 4: 'gozi', 5: 'qbot', 6: 'shade', 7: 'tofsee', 8: 'ursnif'}


In [19]:
stdScale=StandardScaler()
X_train=stdScale.fit_transform(X_train)
X_test=stdScale.fit_transform(X_test)
X_validation=stdScale.fit_transform(X_validation)

In [20]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_validation = X_validation.reshape(X_validation.shape[0], X_validation.shape[1], 1)

In [21]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## **1DCNN**

In [22]:
def CNN(filters=32, kernel_size=3, activation='relu', optimizer='adam', Dropout=0.5):
    model = keras.Sequential()
    model.add(layers.Conv1D(filters=filters, kernel_size=kernel_size, activation=activation, input_shape=(X_train.shape[1], 1)))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=kernel_size, activation=activation))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(Dropout))
    model.add(layers.Dense(128, activation=activation))
    model.add(layers.Dense(9, activation='softmax'))  # Adjust based on number of classes

    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
CNN_model=CNN()
Train=CNN_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_validation, y_validation))

Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 97s 916ms/step - accuracy: 0.7897 - loss: 0.7706 - val_accuracy: 0.9541 - val_loss: 0.2431
Epoch 2/10


KeyboardInterrupt: 

# **Validation evaluation**

In [ ]:
val_accuracy = CNN_model.evaluate(X_validation, y_validation)
print("Validation accuracy=", val_accuracy)

# **Testing**

In [ ]:
y_pred=CNN_model.predict(X_test);

y_pred=np.argmax(y_pred,axis=1)

y_pred=y_pred.reshape(-1,1)

accuracy_before_tuining=accuracy_score(y_test,y_pred)
print(f'Accuracy before tuning: {accuracy_before_tuining}')

In [ ]:
report=classification_report(y_test,y_pred)
print(report)

In [ ]:
# prompt: plot learning curve

import matplotlib.pyplot as plt

plt.plot(Train.history['accuracy'])
plt.plot(Train.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(Train.history['loss'])
plt.plot(Train.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Assuming y_test contains the x-values and you want to plot them against their index:
plt.scatter(range(len(y_test)), y_test, color='blue', label='Actual')

# Assuming y_pred contains the predicted values corresponding to y_test:
plt.scatter(range(len(y_pred)), y_pred, color='red', label='Predicted')

plt.xlabel('Data Point Index/X-axis') # Changed to be more descriptive given the change
plt.ylabel('Values') # Changed to a more generic label that fits both actual and predicted
plt.title('Actual vs Predicted Values') # Adding a title for better clarity
plt.legend()
plt.show()

## **Finding best parameters**

In [ ]:
def build_model(hp):
    model = keras.Sequential()

    filters = hp.Int('filters', min_value=16, max_value=128, step=16)
    kernel_size = hp.Choice('kernel_size', values=[3, 5, 7])
    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.6, step=0.1)
    activation = hp.Choice('activation', values=['relu', 'tanh'])
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    model.add(layers.Conv1D(filters=filters, kernel_size=kernel_size, activation=activation, input_shape=(X_train.shape[1], 1)))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=kernel_size, activation=activation))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(128, activation=activation))
    model.add(layers.Dense(9, activation='softmax'))  # Adjust based on number of classes

    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model






In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Adjust based on the number of trials you want
    executions_per_trial=2,
    directory='my_dir',
    project_name='cnn_tuning'
)

In [ ]:
tuner.search(X_train, y_train, epochs=10, validation_split=0.2)

Reloading Tuner from my_dir/cnn_tuning/tuner0.json

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
80                |16                |filters
7                 |7                 |kernel_size
sigmoid           |sigmoid           |activation
256               |64                |filters_2
0.4               |0.4               |dropout
96                |64                |units
adam              |adam              |optimizer

Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 435s 4s/step - accuracy: 0.3072 - loss: 2.3280 - val_accuracy: 0.3092 - val_loss: 1.9727
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 433s 4s/step - accuracy: 0.3007 - loss: 1.9775 - val_accuracy: 0.3092 - val_loss: 1.9714
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 439s 4s/step - accuracy: 0.3131 - loss: 1.9603 - val_accuracy: 0.3092 - val_loss: 1.9740
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 410s 4s/step - accuracy: 0.2984 - loss: 1.9911 - val_accuracy: 0.3092 - val_loss: 1.9728
Epoch 5/10
103/103 ━━

KeyboardInterrupt: 